# Exploratory Data Analysis

In [1]:
import csv
import pandas as pd

In [14]:
path = 'data/sts-train.csv'

In [15]:
columns=['genre', 'file', 'year', 'index', 'score', 'sentence1', 'sentence2']

In [16]:
df = pd.read_csv(path, sep='\t', usecols=range(7), header=None, quoting=csv.QUOTE_NONE, names=columns, encoding='UTF-8')

In [17]:
df

,genre,file,year,index,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,1,5.00,A plane is taking off.,An air plane is taking off.
1,main-captions,MSRvid,2012test,4,3.80,A man is playing a large flute.,A man is playing a flute.
2,main-captions,MSRvid,2012test,5,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,main-captions,MSRvid,2012test,6,2.60,Three men are playing chess.,Two men are playing chess.
4,main-captions,MSRvid,2012test,9,4.25,A man is playing the cello.,A man seated is playing the cello.
...,...,...,...,...,...,...,...
5744,main-news,headlines,2016,1456,0.00,Severe Gales As Storm Clodagh Hits Britain,Merkel pledges NATO solidarity with Latvia
5745,main-news,headlines,2016,1465,0.00,Dozens of Egyptians hostages taken by Libyan t...,Egyptian boat crash death toll rises as more b...
5746,main-news,headlines,2016,1466,0.00,President heading to Bahrain,President Xi: China to continue help to fight ...
5747,main-news,headlines,2016,1470,0.00,"China, India vow to further bilateral ties",China Scrambles to Reassure Jittery Stock Traders
